In [ ]:
# # # FAILED MODEL FOR INSPECTION TAT (move to next cell)

# import pandas as pd
# import sklearn as sk
# from sklearn.preprocessing import MinMaxScaler
# import datetime
# import matplotlib.pyplot as plt

# # Exporting and filtering raw file

# df_raw=pd.read_csv(r'C:\Users\bhati\Documents\GitHub\DSML\master_notes_files\inspection_data.csv')
# df_raw['INSPECTION_START_TIME']=pd.to_datetime(df_raw.INSPECTION_START_TIME)
# df_raw['INSPECTION_END_TIME']=pd.to_datetime(df_raw.INSPECTION_END_TIME)
# df_raw['INSPECTION_START_DATE']=pd.to_datetime(df_raw.INSPECTION_START_DATE)
# df_raw['INSPECTION_END_DATE']=pd.to_datetime(df_raw.INSPECTION_END_DATE)
# df_raw['inspection_tat']=df_raw['INSPECTION_END_TIME'] - df_raw['INSPECTION_START_TIME']
# df_raw.head()
# df_filtered=df_raw[(df_raw['VERSION']==0) & (df_raw['INSPECTION_STATUS']=='done')]
# df_filtered_2=df_filtered.drop(['VERSION','INSPECTION_STATUS','INSPECTION_START_TIME','INSPECTION_END_TIME','INSPECTION_START_DATE','INSPECTION_END_DATE','APPOINTMENT_ID'],axis=1)

# # target variable encoding

# def target_encoding(df,tar,var):
#     df[var]=df.groupby([var])[tar].transform('mean')
#     return df

# target_encoding(df_filtered_2,'inspection_tat','INSPECTION_BY')
# target_encoding(df_filtered_2,'inspection_tat','INSPECTION_TYPE')
# target_encoding(df_filtered_2,'inspection_tat','CITY')
# target_encoding(df_filtered_2,'inspection_tat','MAKE')
# target_encoding(df_filtered_2,'inspection_tat','MODEL')
# target_encoding(df_filtered_2,'inspection_tat','YEAR')
# target_encoding(df_filtered_2,'inspection_tat','INSPECTION_BY')
# target_encoding(df_filtered_2,'inspection_tat','FUEL_TYPE')

# # Scale dimensions

# from sklearn.preprocessing import MinMaxScaler

# scaler=MinMaxScaler()
# df_scaled=pd.DataFrame(scaler.fit_transform(df_filtered_2),columns=df_filtered_2.columns)

# # splitting dataset into train and test

# from sklearn.model_selection import train_test_split

# y=df_scaled['inspection_tat']
# x=df_scaled.drop('inspection_tat',axis=1)

# x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)

# # Training

# from sklearn.linear_model import LinearRegression

# lm=LinearRegression()
# lm.fit(x_train,y_train)

# # testing scores
# print(lm.score(x_train, y_train))

In [76]:
import datetime as dt
import pandas as pd
import sklearn

In [105]:
# Preprocessing

df_car=pd.read_csv(r"C:\Users\bhati\Documents\GitHub\DSML\13_intro_to_ml_nn\13.2_intro_to_ml_ml_linear_regression_1\cars24-car-price-cleaned.csv")

def target_encoding(df,tar,var):
    df[var]=df.groupby([var])[tar].transform('mean')
    return df

target_encoding(df_car,'selling_price','make')
target_encoding(df_car,'selling_price','model')

# scaling
import sklearn.linear_model
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

df_car_scaled=pd.DataFrame(scaler.fit_transform(df_car),columns=df_car.columns)

# splitting

from sklearn.model_selection import train_test_split

y=df_car_scaled['selling_price']
x=df_car_scaled.drop('selling_price',axis=1)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)

# Training

from sklearn.linear_model import LinearRegression

lm=LinearRegression()

lm.fit(x_train,y_train)

print(lm.score(x_train,y_train))
print(lm.score(x_test,y_test))

0.9458936000772378
0.9459830232581433
